In [1]:
# imports needed packages
from bs4 import BeautifulSoup as bs 
import numpy as np
import pandas as pd
import requests
import re
import xlrd 
import openpyxl

## Population Census DataFrame

In [42]:
#checks site status
url = 'https://www.census.gov/data/tables/time-series/dec/popchange-data-text.html'

req = requests.get(url)

print(req.status_code)


200


In [43]:
soup = bs(req.content, 'html.parser')

In [63]:
def convert(element):
    return element.string if element.string else None

In [64]:
state_names = list(map(convert, soup.select('th')))

In [106]:
#state_names

In [66]:
def remove_until_element(lst, element):
    try:
        index = lst.index(element)
        return lst[index:]
    except ValueError:
        return lst
    
state_names = remove_until_element(state_names, 'United States')

In [67]:
residents = list(map(convert, soup.select('td+ td , #POP_US+ td')))

In [68]:
# Creates two seperate arrays for the resident population and percent change
resident_population = []
percent_change = []

for element in residents:
    
    if '%' in element:
        percent_change.append(element)
    
    else:
        resident_population.append(element)


In [69]:
num_cols = 12
num_rows = 57
reshaped_list = np.array(resident_population).reshape(num_rows, num_cols)

census_resident_pop = pd.DataFrame(reshaped_list, columns=[f'{i}' for i in range(1, num_cols + 1)])

In [70]:
start_year = 2020

for i in range(1, 13):

    original_column_name = str(i)

    census_resident_pop.rename(columns={original_column_name: f'{start_year} Resident Population'}, inplace=True)

    start_year -=10


In [71]:
num_cols = 12
num_rows = 57
reshaped_list = np.array(percent_change).reshape(num_rows, num_cols)

census_percent_change = pd.DataFrame(reshaped_list, columns=[f'{i}' for i in range(1, num_cols + 1)])

start_year = 2020

for i in range(1, 13):

    original_column_name = str(i)

    census_percent_change.rename(columns={original_column_name: f'{start_year} Percentage Change'}, inplace=True)

    start_year -=10



In [72]:
census_df = pd.DataFrame(data=state_names, columns=['State Names'])

In [105]:
#census_df

In [74]:
census_df = census_df.join(census_resident_pop)

**Note:** Due to reasons of data encoding and model construction, the percentage will not be included in the dataframe. In addition to this, certain rows of the dataframe such as the District of Columbia and Puerto Rico will not be included in the dataframe. Puerto Rico for its lack of statehood, and DC due to it already being encompased in a state.

In [76]:
census_df.drop([0, 1, 2, 3, 4, 13, 56], inplace=True)

In [77]:
census_df = census_df.reset_index(drop=True)

In [104]:
#census_df.head()

In [79]:
census_df.to_csv('United_States_Census_Data.csv')

## Fertility DataFrame

In [146]:
fertility_df = pd.read_csv('/Users/emmanuely/Downloads/data-table.csv')

In [147]:
fertility_df.drop(columns=['URL', 'FERTILITY RATE'], axis=1, inplace=True)

In [148]:
fertility_df

,YEAR,STATE,BIRTHS
0,2021,AL,58054
1,2021,AK,9367
2,2021,AZ,77916
3,2021,AR,35965
4,2021,CA,420608
...,...,...,...
445,2005,VA,104555
446,2005,WA,82703
447,2005,WV,20836
448,2005,WI,70984


In [149]:
state_abbr = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
        'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
        'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
        'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
        'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
        'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
        'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
        'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
    }

def switch_state_names(state):
    if state in state_abbr:
        state = state_abbr[state]
        return
    

fertility_df['STATE'] = fertility_df['STATE'].map(state_abbr)


In [150]:
fertility_df = fertility_df.reindex(columns=['STATE', 'YEAR', 'BIRTHS'])

In [151]:
fertility_df = fertility_df.pivot(index='STATE', columns='YEAR', values='BIRTHS').reset_index()

In [152]:
new_columns = {'YEAR':None, 'STATE':'State Names', 2005: '2005 BIRTHS', 2014: '2014 BIRTHS', 
               2015: '2015 BIRTHS', 2016: '2016 BIRTHS', 2017: '2017 BIRTHS', 
               2018: '2018 BIRTHS', 2019: '2019 BIRTHS', 2020: '2020 BIRTHS', 
               2021: '2021 BIRTHS'}

In [153]:
fertility_df.rename(columns=new_columns, inplace=True)

In [155]:
#The isolated year of 2005 can drastically affect our time-series model, hence my decision to remove it
fertility_df.drop(columns=['2005 BIRTHS'], axis=1, inplace=True)

In [157]:
# fertility_df.head()

In [158]:
fertility_df.to_csv('United_States_Fertility_Rates.csv')